In [49]:
# imports
import pandas as pd
import numpy as np

# imports for file logistics
from google.cloud import storage
import gcloud_storage
import os

# imports for image processing
from deephistopath.wsi import slide, tiles
from deephistopath.wsi import filter as filter_

# garbage collector to free memory
import gc

In [2]:
# # SET PATHS
# diagnosis_df_path = './diagnosis_df.csv'
# svs_path = './data/svs_files'
# tiles_png_path = './data/tiles_png/'
# google_credentials_path = './Capstone-Image-Classifcation-393c7bd67e84.json'

# # Establish Credentals
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_credentials_path

In [5]:
diagnosis_df = pd.read_csv(diagnosis_df_path)
diagnosis_df.head(1)

,case_id,file_id,file_name,diagnoses_primary_diagnosis,file_size
0,912d6455-0a50-4fde-b785-f6c09b00b7e4,5df8c657-36b5-410e-a116-cb1f4db29ec7,TCGA-HT-7882-01Z-00-DX4.8B139B5B-567F-4810-A03...,"Oligodendroglioma, anaplastic",561683973


In [52]:
# SET PATHS
diagnosis_df_path = './diagnosis_df.csv'
svs_path = './data/svs_files'
tiles_png_path = './data/tiles_png/'
google_credentials_path = './Capstone-Image-Classifcation-393c7bd67e84.json'

# Establish Credentals
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_credentials_path

# Load in Dataframe
diagnosis_df = pd.read_csv(diagnosis_df_path)

# loop through database at increments = increments
start = 0
increment = 3
list_to_loop = [x for x in range(len(diagnosis_df['file_name']))]

# for each batch of svs files
for i in list_to_loop[start::increment]:

    # for each file in batch:
    for j in range(increment):
        if i+j in list_to_loop:
            # download our svs file to ./data/svs_files/TEST-TR-{slide_num} (needed naming convetion for deephistopath)
            gcloud_storage.download_blob('svs_images_bucket',
                                         diagnosis_df['file_name'][i+j],
                                         os.path.join('data', 'svs_files', f'TEST-TR-{str(j+1).zfill(3)}.svs'))
            
            # add refrences to diagnosis_df
            slide_ref_mask = diagnosis_df['file_name'] == diagnosis_df['file_name'][i+j]
            diagnosis_df.loc[slide_ref_mask, 'batch_number'] = f'batch_{i}'
            diagnosis_df.loc[slide_ref_mask, 'slide_number'] = f'{str(j+1).zfill(3)}'
    
    diagnosis_df.to_csv('./data/diagnosis_df_backup.csv', index = False)
    # run the following on whole batch
    slide.multiprocess_training_slides_to_images()
    filter_.multiprocess_apply_filters_to_images()
    tiles.multiprocess_filtered_images_to_tiles()
    
    # transfer all files (except svs) to bucket to save
    #remove svs first:
    for svs in os.listdir(os.path.join('data', 'svs_files')):
        kill_command = f'rm -rf ./data/svs_files/{svs}'
        os.system(kill_command)

    # copy everything else to bucket
    copy_command_string = f'gsutil -m cp -r ./data gs://test-bucket-capstone/batch_{i}'
    os.system(copy_command_string)
    
    # clean data folder
    for files in os.listdir(os.path.join('data')):
        if files != 'svs_files':
            kill_command = f'rm -rf ./data/{files}'
            os.system(kill_command)
    
    # clear garbage
    for i in range(2):
        n = gc.collect()
        print(f'Unreachable Trash: {n}')

Blob TCGA-HT-7882-01Z-00-DX4.8B139B5B-567F-4810-A03E-4818F3FFFACF.svs downloaded to data/svs_files/TEST-TR-001.svs.
Blob TCGA-HT-7882-01Z-00-DX1.B5224DF8-94F3-47F3-99F0-9D05DEB3B4BC.svs downloaded to data/svs_files/TEST-TR-002.svs.
Blob TCGA-HT-7882-01Z-00-DX3.E16AFAA3-F6A2-45D6-862E-E41AC8E4299F.svs downloaded to data/svs_files/TEST-TR-003.svs.
Opening Slide #1: ./data/svs_files/TEST-TR-001.svs
Opening Slide #2: ./data/svs_files/TEST-TR-002.svs
Opening Slide #3: ./data/svs_files/TEST-TR-003.svs
Number of processes: 3
Number of training images: 3
Task #1: Process slide 1
Task #2: Process slide 2
Task #3: Process slide 3
Saving image to: ./data/training_png/TEST-TR-001-32x-57768x69640-1805x2176.png
Saving image to: ./data/training_png/TEST-TR-003-32x-65736x74157-2054x2317.png
Saving image to: ./data/training_png/TEST-TR-002-32x-69720x72398-2178x2262.png
Done converting slide 1
Done converting slide 2
Done converting slide 3
Time elapsed: 0:00:08.893349
Applying filters to images (multip

Save Thumbnail       | Time: 0:00:00.074987  Name: ./data/filter_thumbnail_jpg/TEST-TR-001-008-rgb-not-green-not-gray-no-pens-remove-small.jpg
Save Image           | Time: 0:00:02.121156  Name: ./data/filter_png/TEST-TR-003-007-rgb-no-gray-no-green-no-pens.png
Save Thumbnail       | Time: 0:00:00.076972  Name: ./data/filter_thumbnail_jpg/TEST-TR-003-007-rgb-no-gray-no-green-no-pens.jpg
Remove Small Objs    | Time: 0:00:00.391428  Type: bool    Shape: (2317, 2054)
Mask RGB             | Time: 0:00:00.033371  Type: uint8   Shape: (2317, 2054, 3)
Save Image           | Time: 0:00:03.598505  Name: ./data/filter_png/TEST-TR-002-006-rgb-no-blue-pen.png
Save Thumbnail       | Time: 0:00:00.113803  Name: ./data/filter_thumbnail_jpg/TEST-TR-002-006-rgb-no-blue-pen.jpg
Mask RGB             | Time: 0:00:00.082532  Type: uint8   Shape: (2262, 2178, 3)
Save Image           | Time: 0:00:02.263370  Name: ./data/filter_png/TEST-TR-001-32x-57768x69640-1805x2176-filtered.png
Save Thumbnail       | Time:

Save Tile            | Time: 0:00:00.101559  Name: ./data/tiles_png/001/TEST-TR-001-tile-r142-c36-x8961-y36100-w256-h256.png
Save Tile            | Time: 0:00:00.135643  Name: ./data/tiles_png/001/TEST-TR-001-tile-r148-c37-x9217-y37636-w256-h256.png
Save Tile            | Time: 0:00:00.125165  Name: ./data/tiles_png/001/TEST-TR-001-tile-r141-c41-x10241-y35844-w256-h256.png
Save Tile            | Time: 0:00:00.105429  Name: ./data/tiles_png/001/TEST-TR-001-tile-r139-c36-x8961-y35332-w256-h256.png
RGB                  | Time: 0:00:00.073229  Type: uint8   Shape: (2620, 2056, 3)
Save Tile            | Time: 0:00:00.127227  Name: ./data/tiles_png/001/TEST-TR-001-tile-r149-c38-x9473-y37892-w256-h256.png
Save Tile            | Time: 0:00:00.120922  Name: ./data/tiles_png/001/TEST-TR-001-tile-r145-c36-x8961-y36868-w256-h256.png
Save Tile            | Time: 0:00:00.144457  Name: ./data/tiles_png/001/TEST-TR-001-tile-r131-c37-x9217-y33284-w256-h256.png
Save Tile            | Time: 0:00:00.09824

Save Tile            | Time: 0:00:00.068243  Name: ./data/tiles_png/001/TEST-TR-001-tile-r137-c96-x24323-y34820-w256-h256.png
Save Tile            | Time: 0:00:00.143089  Name: ./data/tiles_png/001/TEST-TR-001-tile-r122-c37-x9217-y30980-w256-h256.png
Save Tile            | Time: 0:00:00.103570  Name: ./data/tiles_png/001/TEST-TR-001-tile-r140-c38-x9473-y35588-w256-h256.png
RGB                  | Time: 0:00:00.006199  Type: uint8   Shape: (1620, 482, 3)
Save Tile            | Time: 0:00:00.117062  Name: ./data/tiles_png/001/TEST-TR-001-tile-r115-c39-x9729-y29187-w256-h256.png
RGB                  | Time: 0:00:00.127742  Type: uint8   Shape: (2564, 2184, 3)
Save Tile            | Time: 0:00:00.146083  Name: ./data/tiles_png/001/TEST-TR-001-tile-r137-c35-x8705-y34820-w256-h256.png
Save Tile            | Time: 0:00:00.079667  Name: ./data/tiles_png/001/TEST-TR-001-tile-r125-c40-x9985-y31748-w256-h256.png
Save Tile            | Time: 0:00:00.087358  Name: ./data/tiles_png/001/TEST-TR-001-ti

Save Tile            | Time: 0:00:00.058524  Name: ./data/tiles_png/003/TEST-TR-003-tile-r131-c72-x18178-y33286-w256-h256.png
Save Tile            | Time: 0:00:00.101358  Name: ./data/tiles_png/003/TEST-TR-003-tile-r131-c70-x17666-y33286-w256-h256.png
Save Tile            | Time: 0:00:00.054082  Name: ./data/tiles_png/003/TEST-TR-003-tile-r74-c109-x27651-y18691-w256-h256.png
Save Tile            | Time: 0:00:00.057179  Name: ./data/tiles_png/003/TEST-TR-003-tile-r131-c71-x17922-y33286-w256-h256.png
Save Tile            | Time: 0:00:00.074923  Name: ./data/tiles_png/003/TEST-TR-003-tile-r126-c66-x16642-y32006-w256-h256.png
Save Top Orig        | Time: 0:00:04.215274  Name: ./data/top_tile_summary_on_original_png/TEST-TR-002-32x-69720x72398-2178x2262-top_tile_summary.png
Save Tile            | Time: 0:00:00.055069  Name: ./data/tiles_png/003/TEST-TR-003-tile-r95-c79-x19970-y24068-w256-h256.png
Save Top Orig Thumb  | Time: 0:00:00.120390  Name: ./data/top_tile_summary_on_original_thumbnai

...

Process ForkPoolWorker-18:
Process ForkPoolWorker-17:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-19:
Process ForkPoolWorker-24:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-20:
Process ForkPoolWorker-16:
Process ForkPoolWorker-9:
Process ForkPoolWorker-8:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **se

  File "/Users/alex/ga-dsi-11/CAPSTONE/deploy_image_processing/deephistopath/wsi/filter.py", line 421, in filter_rgb_to_hsv
    hsv = sk_color.rgb2hsv(np_img)
KeyboardInterrupt
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/alex/ga-dsi-11/CAPSTONE/deploy_image_processing/deephistopath/wsi/filter.py", line 421, in filter_rgb_to_hsv
    hsv = sk_color.rgb2hsv(np_img)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py", line 276, in rgb2hsv
    idx = (arr[..., 2] == out_v)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/site-p

  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
KeyboardInterrupt
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py",

KeyboardInterrupt: 

  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__

In [56]:
increment = 12

for i in list_to_loop[start::increment]:
    # for each file in batch:
    for j in range(increment):
        if i+j in list_to_loop:
            print(i, j, i + j)

0 0 0
0 1 1
0 2 2
0 3 3
0 4 4
0 5 5
0 6 6
0 7 7
0 8 8
0 9 9
0 10 10
0 11 11
12 0 12
12 1 13
12 2 14
12 3 15
12 4 16
12 5 17
12 6 18
12 7 19
12 8 20
12 9 21
12 10 22
12 11 23
24 0 24
24 1 25
24 2 26
24 3 27
24 4 28
24 5 29
24 6 30
24 7 31
24 8 32
24 9 33
24 10 34
24 11 35
36 0 36
36 1 37
36 2 38
36 3 39
36 4 40
36 5 41
36 6 42
36 7 43
36 8 44
36 9 45
36 10 46
36 11 47
48 0 48
48 1 49
48 2 50
48 3 51
48 4 52
48 5 53
48 6 54
48 7 55
48 8 56
48 9 57
48 10 58
48 11 59
60 0 60
60 1 61
60 2 62
60 3 63
60 4 64
60 5 65
60 6 66
60 7 67
60 8 68
60 9 69
60 10 70
60 11 71
72 0 72
72 1 73
72 2 74
72 3 75
72 4 76
72 5 77
72 6 78
72 7 79
72 8 80
72 9 81
72 10 82
72 11 83
84 0 84
84 1 85
84 2 86
84 3 87
84 4 88
84 5 89
84 6 90
84 7 91
84 8 92
84 9 93
84 10 94
84 11 95
96 0 96
96 1 97
96 2 98
96 3 99
96 4 100
96 5 101
96 6 102
96 7 103
96 8 104
96 9 105
96 10 106
96 11 107
108 0 108
108 1 109
108 2 110
108 3 111
108 4 112
108 5 113
108 6 114
108 7 115
108 8 116
108 9 117
108 10 118
108 11 119
120 0 120


852 11 863
864 0 864
864 1 865
864 2 866
864 3 867
864 4 868
864 5 869
864 6 870
864 7 871
864 8 872
864 9 873
864 10 874
864 11 875
876 0 876
876 1 877
876 2 878
876 3 879
876 4 880
876 5 881
876 6 882
876 7 883
876 8 884
876 9 885
876 10 886
876 11 887
888 0 888
888 1 889
888 2 890
888 3 891
888 4 892
888 5 893
888 6 894
888 7 895
888 8 896
888 9 897
888 10 898
888 11 899
900 0 900
900 1 901
900 2 902
900 3 903
900 4 904
900 5 905
900 6 906
900 7 907
900 8 908
900 9 909
900 10 910
900 11 911
912 0 912
912 1 913
912 2 914
912 3 915
912 4 916
912 5 917
912 6 918
912 7 919
912 8 920
912 9 921
912 10 922
912 11 923
924 0 924
924 1 925
924 2 926
924 3 927
924 4 928
924 5 929
924 6 930
924 7 931
924 8 932
924 9 933
924 10 934
924 11 935
936 0 936
936 1 937
936 2 938
936 3 939
936 4 940
936 5 941
936 6 942
936 7 943
936 8 944
936 9 945
936 10 946
936 11 947
948 0 948
948 1 949
948 2 950
948 3 951
948 4 952
948 5 953
948 6 954
948 7 955
948 8 956
948 9 957
948 10 958
948 11 959
960 0 960
960

1668 5 1673
1668 6 1674
1668 7 1675
1668 8 1676
1668 9 1677
1668 10 1678
1668 11 1679
1680 0 1680
1680 1 1681
1680 2 1682
1680 3 1683
1680 4 1684
1680 5 1685
1680 6 1686
1680 7 1687
1680 8 1688
1680 9 1689
1680 10 1690
1680 11 1691
1692 0 1692
1692 1 1693
1692 2 1694
1692 3 1695
1692 4 1696
1692 5 1697
1692 6 1698
1692 7 1699
1692 8 1700
1692 9 1701
1692 10 1702
